# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 11:53AM,243457,20,ALU0007457,Alumier Labs Inc.,Released
1,Aug 4 2022 11:53AM,243457,20,ALU0007458,Alumier Labs Inc.,Released
2,Aug 4 2022 11:53AM,243457,20,ALUR744091525,Alumier Labs Inc.,Released
3,Aug 4 2022 11:53AM,243457,20,ALUR402721020,Alumier Labs Inc.,Released
4,Aug 4 2022 11:53AM,243457,20,ALUR963540105,Alumier Labs Inc.,Released
5,Aug 4 2022 11:53AM,243457,20,ALUR473730625,Alumier Labs Inc.,Released
6,Aug 4 2022 11:53AM,243457,20,ALUR477827974,Alumier Labs Inc.,Released
7,Aug 4 2022 11:53AM,243457,20,ALUR541566040,Alumier Labs Inc.,Released
8,Aug 4 2022 11:53AM,243457,20,ALUR548849837,Alumier Labs Inc.,Released
9,Aug 4 2022 11:53AM,243457,20,ALU0007456,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,243453,Released,15
19,243454,Released,1
20,243455,Released,1
21,243456,Released,13
22,243457,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243453,NaN,NaN,15.0
243454,NaN,NaN,1.0
243455,NaN,NaN,1.0
243456,NaN,NaN,13.0
243457,NaN,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243414,0.0,0.0,1.0
243415,0.0,0.0,1.0
243416,17.0,27.0,4.0
243417,0.0,2.0,7.0
243428,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243414,0,0,1
243415,0,0,1
243416,17,27,4
243417,0,2,7
243428,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243414,0,0,1
1,243415,0,0,1
2,243416,17,27,4
3,243417,0,2,7
4,243428,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243414,,,1
1,243415,,,1
2,243416,17,27,4
3,243417,,2,7
4,243428,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.
18,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.
19,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.
34,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.
45,Aug 4 2022 11:35AM,243456,10,Beach Products Inc
58,Aug 4 2022 11:33AM,243455,10,Emerginnova
59,Aug 4 2022 11:20AM,243451,19,"AdvaGen Pharma, Ltd"
64,Aug 4 2022 10:33AM,243447,10,Eywa Pharma Inc.
102,Aug 4 2022 10:23AM,243445,19,"AdvaGen Pharma, Ltd"
106,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,,,18
1,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,,,1
2,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,,,15
3,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,,,11
4,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,,,13
5,Aug 4 2022 11:33AM,243455,10,Emerginnova,,,1
6,Aug 4 2022 11:20AM,243451,19,"AdvaGen Pharma, Ltd",,,5
7,Aug 4 2022 10:33AM,243447,10,Eywa Pharma Inc.,,23,15
8,Aug 4 2022 10:23AM,243445,19,"AdvaGen Pharma, Ltd",,,4
9,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,18,,
1,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,1,,
2,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,15,,
3,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,11,,
4,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,13,,
5,Aug 4 2022 11:33AM,243455,10,Emerginnova,1,,
6,Aug 4 2022 11:20AM,243451,19,"AdvaGen Pharma, Ltd",5,,
7,Aug 4 2022 10:33AM,243447,10,Eywa Pharma Inc.,15,23,
8,Aug 4 2022 10:23AM,243445,19,"AdvaGen Pharma, Ltd",4,,
9,Aug 4 2022 9:45AM,243441,10,Acute Outpatient Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,18,,
1,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,1,,
2,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,15,,
3,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,11,,
4,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,18.0,NaN,NaN
1,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,1.0,NaN,NaN
2,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,15.0,NaN,NaN
3,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,11.0,NaN,NaN
4,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 11:53AM,243457,20,Alumier Labs Inc.,18.0,0.0,0.0
1,Aug 4 2022 11:40AM,243454,20,Alumier Labs Inc.,1.0,0.0,0.0
2,Aug 4 2022 11:38AM,243453,20,Alumier Labs Inc.,15.0,0.0,0.0
3,Aug 4 2022 11:36AM,243452,20,Alumier Labs Inc.,11.0,0.0,0.0
4,Aug 4 2022 11:35AM,243456,10,Beach Products Inc,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1947523,114.0,23.0,0.0
12,243440,0.0,1.0,0.0
16,243428,2.0,0.0,0.0
19,973729,20.0,29.0,17.0
20,973816,45.0,0.0,0.0
21,243414,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1947523,114.0,23.0,0.0
1,12,243440,0.0,1.0,0.0
2,16,243428,2.0,0.0,0.0
3,19,973729,20.0,29.0,17.0
4,20,973816,45.0,0.0,0.0
5,21,243414,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,114.0,23.0,0.0
1,12,0.0,1.0,0.0
2,16,2.0,0.0,0.0
3,19,20.0,29.0,17.0
4,20,45.0,0.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,114.0
1,12,Released,0.0
2,16,Released,2.0
3,19,Released,20.0
4,20,Released,45.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,17.0,0.0,0.0
Executing,23.0,1.0,0.0,29.0,0.0,0.0
Released,114.0,0.0,2.0,20.0,45.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,23.0,1.0,0.0,29.0,0.0,0.0
2,Released,114.0,0.0,2.0,20.0,45.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,23.0,1.0,0.0,29.0,0.0,0.0
2,Released,114.0,0.0,2.0,20.0,45.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()